In [1]:
import torch
import numpy as np
from scipy.special import erfc


def erfc1(x):
    tmp = x.cpu().detach().numpy()
    z = erfc(tmp)
    z = torch.from_numpy(z)
    return z

### 公式16

In [15]:
C0 =9
ws=0.0001
ds = 0.0001
temp1 = lambda x, t: torch.div(x - ws * t, torch.sqrt(4 * ds * t))
temp2 = lambda x, t: torch.div(x + ws * t, torch.sqrt(4 * ds * t))
temp10 = lambda x, t: torch.exp(- ws * t/ds)
temp11 = lambda x, t: 0.5 * C0 * (erfc1(temp2(x, t)) + torch.mul(temp10(x, t), erfc1(temp1(x, t))))

In [8]:
region_l = 0
region_r = 1
init_time = 0
end_time = 1
test

In [17]:
def gen_testdata(region_l, region_r, init_time, end_time, test_bach_size):
    test_xy_bach_x = np.linspace(region_l, region_r, test_bach_size).reshape(-1, 1)
    test_xy_bach_y = np.linspace(init_time, end_time, test_bach_size).reshape(-1, 1)
    x_repeat = np.repeat(test_xy_bach_x, test_bach_size).reshape(-1, 1)
    t2 = list(test_xy_bach_y)
    t1 = list(test_xy_bach_y)
    for i in range(test_bach_size - 1):
        t2.extend(t1)
    t_repeat = np.array(t2)
    test_xy_bach = np.concatenate([x_repeat, t_repeat], -1)
    test_xy_torch = torch.from_numpy(test_xy_bach)
    test_x_torch = torch.from_numpy(x_repeat)
    test_y_torch = torch.from_numpy(t_repeat)
    return test_x_torch, test_y_torch

In [18]:
x_repeat, t_repeat = gen_testdata(region_l = region_l, region_r=region_r, init_time=init_time, end_time=end_time, test_bach_size=100)

In [24]:
x_repeat10, t_repeat100 = gen_testdata(region_l = region_l, region_r=10, init_time=init_time, end_time=100, test_bach_size=100)

In [19]:
temp11(x_repeat,t_repeat)

tensor([[   nan],
        [8.9547],
        [8.9099],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]], dtype=torch.float64)

In [25]:
temp11(x_repeat10,t_repeat100)

tensor([[   nan],
        [6.1226],
        [5.0655],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]], dtype=torch.float64)

In [46]:
erfc(0)

1.0

### 论文的解

In [35]:
        pi = np.pi
        # A3的第一项
        temp1 = lambda x, t: torch.div(x - ws * t, torch.sqrt(4 * ds * t))
        temp2 = lambda x, t: torch.div(x + ws * t, torch.sqrt(4 * ds * t))
        temp3 = lambda x, t: 0.5 * erfc1(temp1(x, t)) + 0.5 * torch.exp(ws * x / ds) * erfc1(temp2(x, t))

        # A3的第二项
        temp4 = lambda x, t: 2 + (ws/ds) * (2*region_r-x) + (ws*ws/ds) * t
        temp5 = lambda x, t: np.exp(ws*region_r/ds) * erfc1(torch.div(2*region_r-x+ws*t,torch.sqrt(4*ds*t)))
        temp6 = lambda x, t: 0.5 * torch.mul(temp4(x, t), temp5(x, t))

        # A3的第三项
        temp7 = lambda x, t: ws * torch.sqrt(t/(pi*ds))
        temp8 = lambda x, t: torch.mul(1/(4*ds*t), torch.square(2*region_r- x + ws * t))
        temp9 = lambda x, t: torch.mul(temp7(x, t), torch.exp(ws*region_r/ds - temp8(x, t)))
        Axt = lambda x, t: temp3(x, t) + temp6(x, t) + temp9(x, t)

In [31]:
u_true = lambda x, t: Ci + (C0 - Ci) * Axt(x, t)

In [32]:
Ci=0
C0=8

In [41]:
u_01=u_true(x_repeat,t_repeat)

In [39]:
u_011 = u_true(x_repeat10,t_repeat100)

In [44]:
sum(u_01 == u_011)

tensor([133])

In [45]:
len(u_01)

10000

In [9]:
test_xy_bach_x = np.linspace(region_l, region_r, test_bach_size).reshape(-1, 1)
test_xy_bach_y = np.linspace(init_time, end_time, test_bach_size).reshape(-1, 1)
x_repeat = np.repeat(test_xy_bach_x, test_bach_size).reshape(-1, 1)
t2 = list(test_xy_bach_y)
t1 = list(test_xy_bach_y)
for i in range(test_bach_size - 1):
    t2.extend(t1)
t_repeat = np.array(t2)
test_xy_bach = np.concatenate([x_repeat, t_repeat], -1)

NameError: name 'test_bach_size' is not defined

In [5]:
        region_l = 0.0
        region_r = 10.0
        init_time = 0.0
        end_time = 5
        ds = 1 / 128
        ws = 1
        Ci = 1
        C0 = 2
        temp1 = lambda x, t: torch.div(x - ws * t, torch.sqrt(4 * ds * t))
        temp2 = lambda x, t: torch.div(x + ws * t, torch.sqrt(4 * ds * t))
        temp3 = lambda x, t: 0.5 * erfc1(temp1(x, t)) + 0.5 * torch.exp(ws * x / ds) * erfc1(temp2(x, t))
        u_init = lambda x, t: torch.ones_like(x) * Ci
        u_left = lambda x, t: torch.ones_like(t) * C0
        u_right = lambda x, t: torch.zeros_like(t)
        u_true = lambda x, t: Ci + (Ci - C0) * temp3(x, t)

In [16]:
test_bach_size = 100
size2test = 100

# ---------------------------------------------------------------------------------#
test_xy_bach_x = np.linspace(region_l, region_r, test_bach_size).reshape(-1, 1)
test_xy_bach_y = np.linspace(init_time, end_time, test_bach_size).reshape(-1, 1)
x_repeat = np.repeat(test_xy_bach_x, test_bach_size).reshape(-1, 1)
t2 = list(test_xy_bach_y)
t1 = list(test_xy_bach_y)
for i in range(test_bach_size - 1):
    t2.extend(t1)
    t_repeat = np.array(t2)
test_xy_bach = np.concatenate([x_repeat, t_repeat], -1)

In [22]:
test_xy_torch = torch.from_numpy(test_xy_bach)
x_repeat = torch.from_numpy(x_repeat)
t_repeat = torch.from_numpy(t_repeat)

In [28]:
a=u_true(x_repeat,t_repeat)

In [31]:
UNN2test = torch.nan_to_num(input=a, nan=0.0)

In [48]:
xt = torch.cat([x_repeat,t_repeat],-1)

In [1]:
import torch

In [3]:
a = torch.linspace(0,10,10)

In [4]:
a/abs(a)

tensor([nan, 1., 1., 1., 1., 1., 1., 1., 1., 1.])